In [1]:
# import sys/os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

# Overview

In the following demos we illustrate how to compute the Dynamic Polar Decomposition (PD) of the gradient of the flow map $ \nabla F_{t_0}^t(\mathbf{x}_0) $ on a general three dimensional unsteady velocity dataset. In our case we chose the ABC which is used throughout the demos on advective barriers in 3D. The notebook is structured as follows:

1. Import data from the file 'ABCunsteady.mat' stored in the folder 'data/ABC'.
<br />
2. Define computational parameters and data.
<br />
3. Interpolate velocity from (discrete) gridded data.
<br />
4. Compute $ \mathbf{\nabla F}_{t_0}^{t}(\mathbf{x}_0), \mathbf{F}_{t_0}^{t}(\mathbf{x}_0), \mathbf{W_{t_0}^{t}(\mathbf{x}_0)}, \mathbf{\overline{W}}(t), \quad \forall t \in [t_0, t_N] $.
<br />
5. DPD of $ \mathbf{\nabla F}_{t_0}^{t_N}(\mathbf{x}_0) $

# Import data

In [2]:
# import scipy
import scipy.io as sio

# Import velocity data from file in data-folder
mat_file = sio.loadmat('../../../data/ABC/ABCunsteady.mat')

U = mat_file['u'] # array (NY, NX, NZ)
V = mat_file['v'] # array (NY, NX, NZ)
W = mat_file['w'] # array (NY, NX, NZ)
x = mat_file['x'] # array (1, NX)
y = mat_file['y'] # array (1, NY)
z = mat_file['z'] # array (1, NZ)
time_data = mat_file['t']

# Computational parameters and data

Here we define the computational parameters and the data.

In [3]:
# import numpy
import numpy as np

# Number of cores for parallel computing
Ncores = 7 # int

# Periodic boundary conditions
periodic_x = True # bool
periodic_y = True # bool
periodic_z = True # bool
periodic = [periodic_x, periodic_y, periodic_z]

# Unsteady velocity field
bool_unsteady = True # bool

# Defined domain
defined_domain = np.isfinite(U).astype(int) # array (NY, NX, NZ)

## Compute meshgrid of dataset
X, Y, Z = np.meshgrid(x, y, z) # array (NY, NX), array (NY, NX, NZ)

## Resolution of meshgrid
dx_data = X[0,1,0]-X[0,0,0] # float
dy_data = Y[1,0,0]-Y[0,0,0] # float
dz_data = Y[0,0,1]-Z[0,0,0] # float

delta = [dx_data, dy_data, dx_data] # list (3, )

# Velocity Interpolation

In order to evaluate the velocity field at arbitrary locations, we interpolate the discrete velocity data. The interpolation with respect to time and space is linear.

In [4]:
# Import interpolation function for unsteady flow field
from ipynb.fs.defs.Interpolant import interpolant_unsteady

# Set nan values to zero (in case there are any) so that we can apply interpolant. 
# Interpolant does not work if the array contains nan values. 
U[np.isnan(U)] = 0
V[np.isnan(V)] = 0
W[np.isnan(W)] = 0

# Interpolate velocity data using cubic spatial interpolation
Interpolant = interpolant_unsteady(X, Y, Z, U, V, W, time_data)

Interpolant_u = Interpolant[0] # RectangularBivariateSpline-object
Interpolant_v = Interpolant[1] # RectangularBivariateSpline-object
Interpolant_w = Interpolant[2] # RectangularBivariateSpline-object

# Compute $ \mathbf{\nabla F}_{t_0}^{t}(\mathbf{x}_0), \mathbf{F}_{t_0}^{t}(\mathbf{x}_0), \mathbf{W_{t_0}^{t}(\mathbf{x}_0)}, \mathbf{\overline{W}}(t), \quad \forall t \in [t_0, t_N] $.

In [5]:
# Import function to compute gradient of flow map
from ipynb.fs.defs.gradient_flowmap import gradient_flowmap

# Import function to compute flow map
from ipynb.fs.defs.integration_dFdt import integration_dFdt

# Import function to compute gradient of velocity
from ipynb.fs.defs.gradient_velocity import gradient_velocity

# import package used to suppress output of integration of flow map. 
from IPython.utils import io

# import package for progress bar
from tqdm.notebook import tqdm

# Initial time
t0 = 0

# Final time
tN = 10

# Time step-size (in days)
dt = .05

# NOTE: For computing the backward trajectories set: tN < t0 and dt < 0.

time = np.arange(t0, tN+dt/2, dt) # len(time) = N

# auxiliary grid
aux_grid = [0.01, 0.01, 0.01]

# initial x/y-coordinate
x0 = 3
y0 = 3.4
z0 = 1

#Initial conditions
X0 = np.array([x0, y0, z0]).reshape(3,-1)

# Compute gradient of flow map
with io.capture_output() as captured:
    gradFmap = gradient_flowmap(time, X0, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady, aux_grid) # array (Nt, 3, 3, 1)
    gradFmap = gradFmap[:,:,:,0]
    
# gradFmap stores the gradient of the flow map for all t in [t0,tN] 
# for the trajectory starting at the initial condition defined above.

# Compute flow map
with io.capture_output() as captured:
    Fmap = integration_dFdt(time, X0, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady)[0] # array (Nt, 3, 3, 1)

# Compute average spin tensor over the whole domain:
W_avg = []

for t in tqdm(time):
    
    grad_vel = gradient_velocity(t, np.array([X[::5,::5,::5].ravel(), Y[::5,::5,::5].ravel(), Z[::5,::5,::5].ravel()]), X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady, aux_grid) # array (Nt, 3, 3, NX*NY*NZ)
    
    W_avg.append(np.mean(grad_vel-grad_vel.transpose(1,0,2), axis = -1))
    
W_avg = np.array(W_avg)
# Compute spin along particle trajectory

W = []
for t in tqdm(range(len(time))):
    
    grad_vel = gradient_velocity(t, np.array([Fmap[t,0,0], Fmap[t,1,0], Fmap[t,2,0]]).reshape(3,-1), X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady, aux_grid) # array (Nt, 3, 3, NX*NY*NZ)
    
    grad_vel = grad_vel[:,:,0]
    
    W.append(grad_vel-grad_vel.transpose())
    
W = np.array(W)

  0%|          | 0/201 [00:00<?, ?it/s]

  0%|          | 0/201 [00:00<?, ?it/s]

# Dynamic Polar Decomposition of $ \nabla F_{t_0}^t(\mathbf{x}_0) $

In [16]:
# import function to compute DPD
from ipynb.fs.defs.DPD import DPD

# import package to print latex statement
from IPython.display import display, Markdown

Theta, Phi, O, M, N = DPD(gradFmap, W_avg, W, dt)

gradFmap_t0_tN = gradFmap[-1,:,:]
O_t0_tN = O[-1,:,:]
M_t0_tN = M[-1,:,:]
N_t0_tN = N[-1,:,:]
Theta_t0_tN = Theta[-1,:,:]
Phi_t0_tN = Phi[-1,:,:]

In [13]:
display(Markdown(
   rf"""
   
   $\nabla \mathbf{{F}}_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {gradFmap_t0_tN[0,0]:5.7} & {gradFmap_t0_tN[0,1]:5.7} & {gradFmap_t0_tN[0,2]:5.7} \\ {gradFmap_t0_tN[1,0]:5.7} & {gradFmap_t0_tN[1,1]:5.7} & {gradFmap_t0_tN[1,2]:5.7} \\ {gradFmap_t0_tN[2,0]:5.7} & {gradFmap_t0_tN[2,1]:5.7} & {gradFmap_t0_tN[2,2]:5.7} \end{{pmatrix}} $
   
   $\mathbf{{O}}_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {O_t0_tN[0,0]:5.7} & {O_t0_tN[0,1]:5.7} & {O_t0_tN[0,2]:5.7} \\ {O_t0_tN[1,0]:5.7} & {O_t0_tN[1,1]:5.7} & {O_t0_tN[1,2]:5.7} \\ {O_t0_tN[2,0]:5.7} & {O_t0_tN[2,1]:5.7} & {O_t0_tN[2,2]:5.7} \end{{pmatrix}} $
   
   $\mathbf{{M}}_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {M_t0_tN[0,0]:5.7} & {M_t0_tN[0,1]:5.7} & {M_t0_tN[0,2]:5.7} \\ {M_t0_tN[1,0]:5.7} & {M_t0_tN[1,1]:5.7} & {M_t0_tN[1,2]:5.7} \\ {M_t0_tN[2,0]:5.7} & {M_t0_tN[2,1]:5.7} & {M_t0_tN[2,2]:5.7} \end{{pmatrix}} $
   
   $\mathbf{{N}}_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {N_t0_tN[0,0]:5.7} & {N_t0_tN[0,1]:5.7} & {N_t0_tN[1,2]:5.7} \\ {N_t0_tN[1,0]:5.7} & {N_t0_tN[1,1]:5.7} & {N_t0_tN[1,2]:5.7} \\ {N_t0_tN[2,0]:5.7} & {N_t0_tN[2,1]:5.7} & {N_t0_tN[2,2]:5.7} \end{{pmatrix}} $
   
   $\mathbf{{\Theta}}_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {Theta_t0_tN[0,0]:5.7} & {Theta_t0_tN[0,1]:5.7} & {Theta_t0_tN[1,2]:5.7} \\ {Theta_t0_tN[1,0]:5.7} & {Theta_t0_tN[1,1]:5.7} & {Theta_t0_tN[1,2]:5.7} \\ {Theta_t0_tN[2,0]:5.7} & {Theta_t0_tN[2,1]:5.7} & {Theta_t0_tN[2,2]:5.7} \end{{pmatrix}} $
   
   $\mathbf{{\Phi}}_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {Phi_t0_tN[0,0]:5.7} & {Phi_t0_tN[0,1]:5.7} & {Phi_t0_tN[1,2]:5.7} \\ {Phi_t0_tN[1,0]:5.7} & {Phi_t0_tN[1,1]:5.7} & {Phi_t0_tN[1,2]:5.7} \\ {Phi_t0_tN[2,0]:5.7} & {Phi_t0_tN[2,1]:5.7} & {Phi_t0_tN[2,2]:5.7} \end{{pmatrix}} $
   
   """))


   
   $\nabla \mathbf{F}_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} -52.69229 & -10.88534 & -23.90324 \\ 25.76554 & 4.563932 & 11.27945 \\ -25.67696 & -5.759289 & -12.52595 \end{pmatrix} $
   
   $\mathbf{O}_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} 8.124543 & 47.32643 & -15.18824 \\ 15.93807 & 12.64054 & -12.97531 \\ 42.37295 & -26.91382 & -25.51698 \end{pmatrix} $
   
   $\mathbf{M}_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} 36.32308 & 7.07615 & 16.35759 \\ 9.21283 & 1.792087 & 4.155168 \\ 51.60644 & 10.08602 & 23.27132 \end{pmatrix} $
   
   $\mathbf{N}_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} 24.80348 & -59.13808 & -7.790425 \\ -11.92058 & 28.40483 & -7.790425 \\ 12.46364 & -29.707 & 8.178187 \end{pmatrix} $
   
   $\mathbf{\Theta}_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} 0.5416825 & -0.4850176 & 0.5075631 \\ 1.034898 & -0.2192335 & 0.5075631 \\ -0.4629994 & -1.105233 & 0.1328513 \end{pmatrix} $
   
   $\mathbf{\Phi}_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} 3.502258 & -12.70033 & -17.14083 \\ 12.44761 & 1.216759 & -17.14083 \\ 40.1903 & 21.04416 & 2.539943 \end{pmatrix} $
   
   

# References

[1] Notebook 2.3.6. in "Transport Barriers and Coherent Structures in Flow Data" by Prof. George Haller.